In [ ]:
# hide
import sys
sys.path.append("..")
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# DCAE Hyperparameter sweep

> Run a hyperparamter sweep for the Deep Convolutional AutoEncoder defined in the notebook `01_Timecluster_replication.ipynb`

This sweep has been run using Weights & Biases, following [this tutorial](https://www.wandb.com/articles/running-hyperparameter-sweeps-to-pick-the-best-model-using-w-b).

In [ ]:
#hide
import wandb
from tensorflow.keras.optimizers import Adam
from wandb.keras import WandbCallback
from pacmel_mining_use_case.load import *
from pacmel_mining_use_case.timecluster import *
from functools import partial
from fastcore.utils import Path

### Sweep configuration 

In [ ]:
# Select the hyperparameters you want to tune. This is specified like the following:
sweep_config = {
   'method': 'random',
    "metric": {
        "name" : "val_loss",
        "goal" : "minimize"
    },
   'parameters': {
       'w': {
           'values': list(range(12, 144, 12))
       },
        'stride': {
           'values': [1, 5, 10, 20]
       },
       'lr': {
           'distribution': 'uniform',
           'min': 0.00001,
           'max': 0.01
       },
       'epochs': {
           'distribution': 'int_uniform',
           'min': 1,
           'max': 50
       },
       'batch_size': {
           'distribution': 'int_uniform',
           'min': 32,
           'max': 256
       },
       'n_filters': {
           'values': [[64,32,12],[32,16,12]]
       },
       'filter_sizes': {
           'values': [[10, 5, 5], [20, 10, 10], [30, 15, 15]]
       },
       'output_filter_size': {
           'distribution': 'int_uniform',
           'min': 5,
           'max': 30
       },
       'pool_sizes': {
           'value': [2, 2, 3]
       },
       'dataset': {
           'value': 'pacmel_mining_dataset:a77b7c7e39fa5564203b74e76fecfb94'
       }
   },
    'early_terminate': {
        'type': 'hyperband',
        's': 2,
        'eta': 3,
        'max_iter': 27
    }
}

### Initialize sweep

In [ ]:
sweep_id = wandb.sweep(sweep_config)

Create sweep with ID: 05f5p9f9
Sweep URL: https://app.wandb.ai/vrodriguezf/timecluster-extension/sweeps/05f5p9f9


### Define the agent

Once you have initialized the sweep you need an agent. An agent is a model training script you can use to pair the sweep configurations. Let’s define a simple training script:fmultiTSloader

In [ ]:
def train():
    # Initialize wandb with a sample project name
    run = wandb.init(project="timecluster-extension", 
                     job_type='train_DCAE', 
                     allow_val_change=True)
    
    # Specify the hyperparameter to be tuned along with an initial value
    config_defaults = {
        'lr': 0.002438113880973422,
        'n_filters': [32, 16, 12],
        'filter_sizes': [10, 5, 5],
        'output_filter_size': 10,
        'pool_sizes': [2, 2, 3],
        'batch_size': 75,
        'epochs': 34,
        'stride': 1,
        'w': 60
    }
    
    # Load and preprocess the data
    ds_artifact = run.use_artifact(type='dataset', 
                                   name=wandb.config.dataset)
    datadir = Path(ds_artifact.download())
    _, _, input_data = fmultiTSloader(df_paths=datadir.ls(), 
                                w=wandb.config.w, 
                                stride=wandb.config.stride, 
                                ts_colname='description')

    # Specify the other hyperparameters to the configuration
    wandb.config.update(allow_val_change=True, 
                        params={
                            'ds_artifact_type_name': ds_artifact.artifact_type_name,
                            'ds_artifact_digest': ds_artifact.digest,
                            'val_pct': 0.2
                        })

    # Define the model
    model = createDCAE(w=wandb.config.w, 
                       d=input_data.shape[2], 
                       delta=wandb.config.w, 
                       n_filters=wandb.config.n_filters, 
                       filter_sizes=wandb.config.filter_sizes, 
                       pool_sizes=wandb.config.pool_sizes, 
                       output_filter_size=wandb.config.output_filter_size)

    opt = Adam(learning_rate=wandb.config.lr)

    # Compile the model
    model.compile(optimizer=opt,
                 loss='mean_squared_error',
                 metrics=['mean_squared_error'])

    # Train the model
    model.fit(x=input_data, y=input_data, batch_size=wandb.config.batch_size, 
      validation_split=wandb.config.val_pct, epochs=wandb.config.epochs, verbose=0, 
      callbacks=[WandbCallback()])

### Run the training script as a wandb agent 

In [ ]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: 6dihclwi with config:
	batch_size: 209
	dataset: pacmel_mining_dataset:a77b7c7e39fa5564203b74e76fecfb94
	epochs: 40
	filter_sizes: [30, 15, 15]
	lr: 0.009735469368300498
	n_filters: [64, 32, 12]
	output_filter_size: 19
	pool_sizes: [2, 2, 3]
	stride: 1
	w: 108
wandb: Agent Started Run: 6dihclwi


wandb: Wandb version 0.8.31 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
../pacmel_mining_use_case/load.py:72: DtypeWarning: Columns (18,19,20,21,22,23,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,93,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  df = fread_mining_monitoring_files(df_paths)
/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


wandb: Agent Finished Run: 6dihclwi 

wandb: Agent Starting Run: gr3ckmc2 with config:
	batch_size: 217
	dataset: pacmel_mining_dataset:a77b7c7e39fa5564203b74e76fecfb94
	epochs: 21
	filter_sizes: [20, 10, 10]
	lr: 0.0035585746874123732
	n_filters: [64, 32, 12]
	output_filter_size: 23
	pool_sizes: [2, 2, 3]
	stride: 20
	w: 84
wandb: Agent Started Run: gr3ckmc2


wandb: Wandb version 0.8.31 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
../pacmel_mining_use_case/load.py:72: DtypeWarning: Columns (18,19,20,21,22,23,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,93,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  df = fread_mining_monitoring_files(df_paths)
/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


wandb: Agent Finished Run: gr3ckmc2 

wandb: Agent Starting Run: n2106bvi with config:
	batch_size: 239
	dataset: pacmel_mining_dataset:a77b7c7e39fa5564203b74e76fecfb94
	epochs: 3
	filter_sizes: [30, 15, 15]
	lr: 0.006982953934503286
	n_filters: [64, 32, 12]
	output_filter_size: 23
	pool_sizes: [2, 2, 3]
	stride: 1
	w: 12
wandb: Agent Started Run: n2106bvi


...

wandb: Agent Finished Run: a20stj7d 

wandb: Agent Starting Run: vdoefngv with config:
	batch_size: 119
	dataset: pacmel_mining_dataset:a77b7c7e39fa5564203b74e76fecfb94
	epochs: 31
	filter_sizes: [10, 5, 5]
	lr: 0.0014586227213578933
	n_filters: [32, 16, 12]
	output_filter_size: 9
	pool_sizes: [2, 2, 3]
	stride: 5
	w: 96
wandb: Agent Started Run: vdoefngv


wandb: Wandb version 0.8.31 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
../pacmel_mining_use_case/load.py:72: DtypeWarning: Columns (18,19,20,21,22,23,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,93,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  df = fread_mining_monitoring_files(df_paths)
/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


wandb: Agent Finished Run: vdoefngv 

wandb: Agent Starting Run: yi3u2ymp with config:
	batch_size: 40
	dataset: pacmel_mining_dataset:a77b7c7e39fa5564203b74e76fecfb94
	epochs: 48
	filter_sizes: [10, 5, 5]
	lr: 0.003415861001137198
	n_filters: [32, 16, 12]
	output_filter_size: 29
	pool_sizes: [2, 2, 3]
	stride: 1
	w: 96
wandb: Agent Started Run: yi3u2ymp


wandb: Wandb version 0.8.31 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
../pacmel_mining_use_case/load.py:72: DtypeWarning: Columns (18,19,20,21,22,23,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,93,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  df = fread_mining_monitoring_files(df_paths)
/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


wandb: Agent Finished Run: yi3u2ymp 

